# 1: Intro to Geospatial tools

This script introduces how to:
- use Geopandas to read
- write shapefiles
- visualize basic GIS vectors
- add interactive basemap visuals for context

In [ ]:
# Base libs
import os
import glob 
import shutil

# Geospatial libs
import geopandas as gpd
import folium

# other libs
import matplotlib.pyplot as plt
%matplotlib inline

# 0. Use Geopandas to read files

Geopandas (gpd) treats spatial data like a data frame (rows x columns) but with a unique structure so they operate very similarly to a pandas dataframe. A gpd geodataframe contains a geometry column specifying a tuple/coordinate. A tuple is a common python data type where multiple values are linked. Comparatively a two-item list is unlinked, i.e. each object in the list can be changed, inserted, or deleted while the tuple cannot be modified in the same way. In the geo-df the tuples define vertices for points, lines, polygons, etc. 

Geospatial vector data are handled by the library shapely, evene in gpd.

In [ ]:
# check where we are 
%pwd

In [ ]:
# I like using glob for pathnames but os provides similar functionality, it can handle relative paths with more ease
shp_files = glob.glob('./data/shapefiles/*')

# use print to display multiple outputs in each cell 
print(shp_files)

# unlike R, python does not need 'paste' to combine multiple vars in a single print statement
# + and , behave slightly different in print statements. Particularly important joining file paths
print('\n' + 'The Sonoma .shp file:', shp_files[5]) 

In [ ]:
# read a shapefile using the .shp extension
temp_sp = gpd.read_file(shp_files[5])
type(temp_sp)

Exploring a geospatial DF struct

In [ ]:
# df.head() is a common pandas tool to see the first 5 rows of a df
# in our case this object has only one row with a polygon object
temp_sp.head()

In [ ]:
# remember python is 0 indexed so our first row, above, is row 0 and the df is length == 1
print(len(temp_sp))

In [ ]:
# subset our df
sonoma = temp_sp[['NAME','AREA_','geometry']]
sonoma

In [ ]:
# visualize the geo-df
sonoma.plot()

# 1. Checking projections

First, checking projections and coordinate reference systems (CRS).
- CRSs locate our objects on earth 
- Geographic CRSs place objects in a coordinate plane and not a plot with a 0,0 origin. Commonly lat/long.
- Projections (projected CRS) transforms lat/long into a planer surface meaning our 3D corrdinates are represented on a 2D plane and have discrete local regions to minimize distortions. Commonly meters.
- In python we will use gpd and pyproj (a dependency of gpd) to project data

In [ ]:
# What are these coordinate values representing?
water_shed = gpd.read_file('./data/shapefiles/HYD_WTRSHD_HUC.shp')
water_shed.head()

In [ ]:
# Let's use our russian river subset
# check our current projection
water_shed.crs

From our CRS we can see an EPSG (European Petroleum Survey Group) code is [4326](https://epsg.io/4326) meaning it uses the WGS84 coordinate system in Lat/Long.

For demonstration, let's reproject to EPSG:26910 or NAD83 UTM zone 10N

In [ ]:
reproj = water_shed.to_crs(epsg = 26910)
reproj.plot()

Notice that the coordinate system has now changed to meters from Lat/Long

In [ ]:
reproj2 = water_shed.to_crs(epsg = 32618)
reproj2.plot()

It can be pretty easy to see when your projection is wrong! A big reason to save your reprojected data to a new variable. If you want to reproject again you may begin to run into issues with conistency. It is best to only project once and be aware if your data is projected to begin with. 

# 2. Manipulate spatial data
### Let's explore another geospatial data type - multi polygon

In [ ]:
water_shed.head()

In [ ]:
# commas allow print statements to concatenate different datatypes
print('Number of points:', len(water_shed))
print('How many different HUC types:', water_shed['Name'].unique())

In [ ]:
# subset to only Russian river watersheds
russian_river = water_shed[water_shed['Name'].str.contains('Russian', na = False)]
russian_river['Name'].unique()

In [ ]:
russian_river.plot()

### Point and polygon intersection

In [ ]:
pts = gpd.read_file('./data/shapefiles/PWD_MetStations2021.shp')
pts.head()

In [ ]:
pts.plot()

In [ ]:
pts.crs

In [ ]:
# Check if our projections are the same!
pts.crs == sonoma.crs

In [ ]:
# reproject sonoma to match gps points
proj = sonoma.to_crs(epsg = 32610)
pts.crs == proj.crs

In [ ]:
proj.contains(pts.iloc[[0]])

# 3. Saving geopandas objects

Save out subselection of the Russian River drainages to a new shapefile

In [ ]:
out_dir = './results'

# specify an out file
out_file = os.path.join(out_dir, "Class_36200.shp")
russian_river.to_file(out_file)

In [ ]:
# read in our selection to check it
temp = gpd.read_file(out_file)
temp.plot()

In [ ]:
# clean up what we created
folder = './results/*'
files = glob.glob(folder)
print(files)
for f in files:  
    os.remove(f)

In [ ]:
glob.glob(folder)

# 4. Basic visualization of geospatial data

In [ ]:
# Make subplots that are next to each other
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(12, 12))

# Plot the original CRS
russian_river.plot(ax=ax1, facecolor='gray')

# Add title
ax1.set_title("WGS84")

# Plot the one with the incorrect projection
reproj2.plot(ax=ax2, facecolor='blue')

# Add title
ax2.set_title("Incorrect projections")

# Set aspect ratio as 1
ax1.set_aspect(aspect=1)
ax2.set_aspect(aspect=1)

# Remove empty white space around the plot
plt.tight_layout()

# 5. Adding basemaps for visualization and context

Folium by default uses OpenStreetMap but also can pull down Stamen Terrain and Stamen Toner. Markers can be added using [bootstrap](https://getbootstrap.com/docs/3.3/components/)

In [ ]:
# Show set zoom for Flagstaff OpenStreetMaps
m = folium.Map(location=[35.1983, -111.6513], zoom_start=14)

# Add markers to LOIs
folium.Marker([35.18619596476278, -111.65855717457414], 
              popup="<i>SICCS</i>", 
              tooltip = "Where am I?",
              icon=folium.Icon(color="red", icon="glyphicon-thumbs-up")
             ).add_to(m)

# display map
m

Sometimes it can be helpful to dipslay coords to recenter a map:

In [ ]:
m = folium.Map(location=[35.1983, -111.6513], zoom_start=11, tiles = 'Stamen Terrain')
m.add_child(folium.LatLngPopup())
m